In [1]:
import geopandas as gpd
import rioxarray as rxr
from rasterstats import zonal_stats
import pandas as pd
import os
import rasterio
from rasterio.mask import mask
import numpy as np
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
from shapely.ops import unary_union
from rasterio.features import rasterize
from data_utils import best_res_align, align_r1_to_r2
import xarray as xr

In [2]:
# laptop folder
folder = "../../_data/p-dengue/"
valid_admin2 = pd.read_csv(os.path.join(folder, 'valid_admin/valid_admin2.csv'), header=None)[0].tolist()
valid_admin1 = pd.read_csv(os.path.join(folder, 'valid_admin/valid_admin1.csv'), header=None)[0].tolist()
valid_admin2.sort()
valid_admin1.sort()

In [3]:
#### shape data
admin1_shp = gpd.read_file(os.path.join(folder,"shapefiles/admin1/admin1_38.shp"))
admin2_shp = gpd.read_file(os.path.join(folder,"shapefiles/admin2/admin2_38.shp"))

admin1_shp = (admin1_shp[admin1_shp["admin1"].isin(valid_admin1)].sort_values("admin1").reset_index(drop=True))
admin2_shp = (admin2_shp[admin2_shp["admin2"].isin(valid_admin2)].sort_values("admin2").reset_index(drop=True))

print(f'shapefile crs: {admin1_shp.crs}')

#### Population raster
raster_pop = {year :  rxr.open_rasterio(os.path.join(folder, f'raster_pop/landscan-global-{year}.tif'), masked=True) for year in range(2015,2025)}
print(f'raster_pop crs: {raster_pop[2015].rio.crs}')

#### Urban raster (GHSL GHS-SMOD)
raster_urbanisation = rxr.open_rasterio(os.path.join(folder, 'raster_urbanisation/GHS_SMOD_E2020_GLOBE_R2023A_54009_1000_V2_0.tif'), masked=True)

raster_urbanisation_binary = raster_urbanisation.copy()
urban_classes = [21, 22, 23, 30]
raster_urbanisation_binary = raster_urbanisation_binary.where(~raster_urbanisation_binary.isin(urban_classes), other=1)
rural_classes = [10, 11, 12, 13]
raster_urbanisation_binary = raster_urbanisation_binary.where(~raster_urbanisation_binary.isin(rural_classes), other=0)
raster_urbanisation_binary = raster_urbanisation_binary.where((raster_urbanisation_binary == 0) | (raster_urbanisation_binary == 1))
print(f'raster_urbanisation crs: {raster_urbanisation.rio.crs}')
raster_urbanisation = raster_urbanisation.rio.reproject(admin1_shp.crs)
raster_urbanisation_binary = raster_urbanisation_binary.rio.reproject(admin1_shp.crs)
print(f'raster_urbanisation crs projected to: {raster_urbanisation.rio.crs}')

#### Surveillance raster (Ahyoung Lim et al. Nature Medicine)
raster_surveillance = rxr.open_rasterio(os.path.join(folder, 'raster_surveillance/Surveillance_map_wmean.tif'), masked=True)
# EPSG:4326
raster_surveillance = raster_surveillance.rio.write_crs("EPSG:4326")
print(f'raster_surveillance crs: {raster_surveillance.rio.crs}')

#### Climate data rasters

shapefile crs: EPSG:4326
raster_pop crs: EPSG:4326
raster_urbanisation crs: ESRI:54009
raster_urbanisation crs projected to: EPSG:4326
raster_surveillance crs: EPSG:4326


In [4]:
merged = admin1_shp.union_all()
merged_buffered = gpd.GeoDataFrame(geometry=[merged], crs=admin1_shp.crs)
merged_buffered["geometry"] = (merged_buffered.to_crs("EPSG:3857").buffer(20_000).to_crs("EPSG:4326"))
region_bounds_buffered = merged_buffered.bounds.iloc[0, :].to_list()

In [5]:
pop_crop_alignment = raster_pop[2015].squeeze().rio.clip_box(minx=region_bounds_buffered[0], miny=region_bounds_buffered[1],
                                                   maxx=region_bounds_buffered[2], maxy=region_bounds_buffered[3])

---

### admin_year_pop

In [5]:
admin1_records = []
admin2_records = []

for year in range(2015,2025):
    pop_crop = raster_pop[year].squeeze().rio.clip_box(minx=region_bounds_buffered[0], miny=region_bounds_buffered[1],
                                                   maxx=region_bounds_buffered[2], maxy=region_bounds_buffered[3])
    admin1_pop_stats = zonal_stats(
        vectors=admin1_shp,
        raster=pop_crop.values,
        affine=pop_crop.rio.transform(),
        stats=["sum"],
        nodata=pop_crop.rio.nodata,
        all_touched=True)
    
    for i, row in enumerate(admin1_shp.itertuples()):
        admin1_records.append({
            "admin1": row.admin1,
            "year": year,
            "population": admin1_pop_stats[i]["sum"]
        })

    admin2_pop_stats = zonal_stats(
        vectors=admin2_shp,
        raster=pop_crop.values,
        affine=pop_crop.rio.transform(),
        stats=["sum"],
        nodata=pop_crop.rio.nodata,
        all_touched=True)
    
    for i, row in enumerate(admin2_shp.itertuples()):
        admin2_records.append({
            "admin2": row.admin2,
            "year": year,
            "population": admin2_pop_stats[i]["sum"]
        })

# Convert to DataFrames
df_admin1_pop = pd.DataFrame(admin1_records)
df_admin2_pop = pd.DataFrame(admin2_records)

# Optional: reset index
df_admin1_pop = df_admin1_pop.reset_index(drop=True)
df_admin2_pop = df_admin2_pop.reset_index(drop=True)

In [6]:
out_folder = os.path.join(folder, "admin_year_pop")
os.makedirs(out_folder, exist_ok=True)

df_admin1_pop.to_csv(os.path.join(out_folder, "admin1_year_pop.csv"), index=False)
df_admin2_pop.to_csv(os.path.join(out_folder, "admin2_year_pop.csv"), index=False)

---

### urbanisation

In [7]:
admin1_records = []
admin2_records = []

# population-aligned urbanisation raster
_, raster_urbanisation_binary_pop_aligned = best_res_align(raster_pop[2015], '',
                                                    raster_urbanisation_binary, 'nearest',
                                                    region_bounds_buffered = region_bounds_buffered,
                                                    shape_crs = admin1_shp.crs)

r1 resolution: 0.008333333333333304 -0.008333333333333297
r2 resolution: 0.009961733526091017 -0.009961733526091015
r2 is lower resolution, downsampling r2 to r1


In [8]:
for year in range(2015,2025):
    pop_crop = raster_pop[year].squeeze().rio.clip_box(minx=region_bounds_buffered[0], miny=region_bounds_buffered[1],
                                                   maxx=region_bounds_buffered[2], maxy=region_bounds_buffered[3])
    # admin1
    # Non-weighted: simple mean over each polygon
    urbanisation_nonweighted = zonal_stats(
        vectors=admin1_shp,
        raster=raster_urbanisation_binary_pop_aligned.values,
        affine=raster_urbanisation_binary_pop_aligned.rio.transform(),
        stats=["mean"],
        nodata=raster_urbanisation_binary_pop_aligned.rio.nodata,
        all_touched=True
    )

    # Population-weighted: sum(urban * pop) / sum(pop)
    urbanisation_pop_weighted_records = []
    for i, row in enumerate(admin1_shp.itertuples()):
        mask = rasterize([(row.geometry, 1)],
                         out_shape=raster_urbanisation_binary_pop_aligned.shape[-2:],
                         transform=raster_urbanisation_binary_pop_aligned.rio.transform(),
                         fill=0,
                         all_touched=True,
                         dtype="uint8"
                         ).astype(bool)
        urbanisation_values = raster_urbanisation_binary_pop_aligned.values[mask]
        pop_values = pop_crop.values[mask]

        # Keep only cells where BOTH are valid
        valid = (~np.isnan(urbanisation_values)) & (~np.isnan(pop_values)) & (pop_values > 0)

        if not np.any(valid):
            weighted_mean = np.nan
        else:
            weighted_mean = np.sum(urbanisation_values[valid] * pop_values[valid]) / np.sum(pop_values[valid])
        urbanisation_pop_weighted_records.append(weighted_mean)

    # Store results in admin1_records
    for i, row in enumerate(admin1_shp.itertuples()):
        admin1_records.append({
            "admin1": row.admin1,
            "year": year,
            "urbanisation_nonweighted": urbanisation_nonweighted[i]["mean"],
            "urbanisation_pop_weighted": urbanisation_pop_weighted_records[i]
        })

    # admin2
    # Non-weighted: simple mean over each polygon
    urbanisation_nonweighted = zonal_stats(
        vectors=admin2_shp,
        raster=raster_urbanisation_binary_pop_aligned.values,
        affine=raster_urbanisation_binary_pop_aligned.rio.transform(),
        stats=["mean"],
        nodata=raster_urbanisation_binary_pop_aligned.rio.nodata,
        all_touched=True
    )

    # Population-weighted: sum(urban * pop) / sum(pop)
    urbanisation_pop_weighted_records = []
    for i, row in enumerate(admin2_shp.itertuples()):
        mask = rasterize([(row.geometry, 1)],
                         out_shape=raster_urbanisation_binary_pop_aligned.shape[-2:],
                         transform=raster_urbanisation_binary_pop_aligned.rio.transform(),
                         fill=0,
                         all_touched=True,
                         dtype="uint8"
                         ).astype(bool)
        urbanisation_values = raster_urbanisation_binary_pop_aligned.values[mask]
        pop_values = pop_crop.values[mask]

        # Keep only cells where BOTH are valid
        valid = (~np.isnan(urbanisation_values)) & (~np.isnan(pop_values)) & (pop_values > 0)

        if not np.any(valid):
            weighted_mean = np.nan
        else:
            weighted_mean = np.sum(urbanisation_values[valid] * pop_values[valid]) / np.sum(pop_values[valid])
        urbanisation_pop_weighted_records.append(weighted_mean)

    # Store results in admin2_records
    for i, row in enumerate(admin2_shp.itertuples()):
        admin2_records.append({
            "admin2": row.admin2,
            "year": year,
            "urbanisation_nonweighted": urbanisation_nonweighted[i]["mean"],
            "urbanisation_pop_weighted": urbanisation_pop_weighted_records[i]
        })

# Convert to DataFrames
df_admin1_urbanisation = pd.DataFrame(admin1_records)
df_admin2_urbanisation = pd.DataFrame(admin2_records)

# Optional: reset index
df_admin1_urbanisation = df_admin1_urbanisation.reset_index(drop=True)
df_admin2_urbanisation = df_admin2_urbanisation.reset_index(drop=True)

In [9]:
df_admin2_urbanisation

,admin2,year,urbanisation_nonweighted,urbanisation_pop_weighted
0,ACEH BARAT,2015,0.018364,0.224615
1,ACEH BARAT DAYA,2015,0.037194,0.666287
2,ACEH BESAR,2015,0.099596,0.589342
3,ACEH JAYA,2015,0.009563,0.256873
4,ACEH SELATAN,2015,0.038308,0.405260
...,...,...,...,...
5135,WAY KANAN,2024,0.045294,0.231300
5136,WONOGIRI,2024,0.453802,0.694331
5137,WONOSOBO,2024,0.513472,0.795368
5138,YAHUKIMO,2024,0.008076,0.069001


In [10]:
out_folder = os.path.join(folder, "admin_year_urbanisation")
os.makedirs(out_folder, exist_ok=True)

df_admin1_urbanisation.to_csv(os.path.join(out_folder, "admin1_year_urbanisation.csv"), index=False)
df_admin2_urbanisation.to_csv(os.path.join(out_folder, "admin2_year_urbanisation.csv"), index=False)

---

### surveillance

In [11]:
admin1_records = []
admin2_records = []

# population-aligned surveillance raster
_, raster_surveillance_pop_aligned = best_res_align(raster_pop[2015], '',
                                                    raster_surveillance, 'nearest',
                                                    region_bounds_buffered = region_bounds_buffered,
                                                    shape_crs = admin1_shp.crs)

r1 resolution: 0.008333333333333304 -0.008333333333333297
r2 resolution: 0.04166666666662877 -0.04166666666665719
r2 is lower resolution, downsampling r2 to r1


In [12]:
for year in range(2015,2025):
    pop_crop = raster_pop[year].squeeze().rio.clip_box(minx=region_bounds_buffered[0], miny=region_bounds_buffered[1],
                                                   maxx=region_bounds_buffered[2], maxy=region_bounds_buffered[3])
    # admin1
    # Non-weighted: simple mean over each polygon
    surveillance_nonweighted = zonal_stats(
        vectors=admin1_shp,
        raster=raster_surveillance_pop_aligned.values,
        affine=raster_surveillance_pop_aligned.rio.transform(),
        stats=["mean"],
        nodata=raster_surveillance_pop_aligned.rio.nodata,
        all_touched=True
    )

    # Population-weighted: sum(surveillance * pop) / sum(pop)
    pop_weighted_records = []
    for i, row in enumerate(admin1_shp.itertuples()):
        mask = rasterize([(row.geometry, 1)],
                         out_shape=raster_surveillance_pop_aligned.shape[-2:],
                         transform=raster_surveillance_pop_aligned.rio.transform(),
                         fill=0,
                         all_touched=True,
                         dtype="uint8"
                         ).astype(bool)
        surveillance_values = raster_surveillance_pop_aligned.values[mask]
        pop_values = pop_crop.values[mask]

        # Keep only cells where BOTH are valid
        valid = (~np.isnan(surveillance_values)) & (~np.isnan(pop_values)) & (pop_values > 0)

        if not np.any(valid):
            weighted_mean = np.nan
        else:
            weighted_mean = np.sum(surveillance_values[valid] * pop_values[valid]) / np.sum(pop_values[valid])
        pop_weighted_records.append(weighted_mean)

    # Store results in admin1_records
    for i, row in enumerate(admin1_shp.itertuples()):
        admin1_records.append({
            "admin1": row.admin1,
            "year": year,
            "surveillance_nonweighted": surveillance_nonweighted[i]["mean"],
            "surveillance_pop_weighted": pop_weighted_records[i]
        })

    # admin2
    # Non-weighted: simple mean over each polygon
    surveillance_nonweighted = zonal_stats(
        vectors=admin2_shp,
        raster=raster_surveillance_pop_aligned.values,
        affine=raster_surveillance_pop_aligned.rio.transform(),
        stats=["mean"],
        nodata=raster_surveillance_pop_aligned.rio.nodata,
        all_touched=True
    )

    # Population-weighted: sum(surveillance * pop) / sum(pop)
    pop_weighted_records = []
    for i, row in enumerate(admin2_shp.itertuples()):
        mask = rasterize([(row.geometry, 1)],
                         out_shape=raster_surveillance_pop_aligned.shape[-2:],
                         transform=raster_surveillance_pop_aligned.rio.transform(),
                         fill=0,
                         all_touched=True,
                         dtype="uint8"
                         ).astype(bool)
        surveillance_values = raster_surveillance_pop_aligned.values[mask]
        pop_values = pop_crop.values[mask]

        # Keep only cells where BOTH are valid
        valid = (~np.isnan(surveillance_values)) & (~np.isnan(pop_values)) & (pop_values > 0)

        if not np.any(valid):
            weighted_mean = np.nan
        else:
            weighted_mean = np.sum(surveillance_values[valid] * pop_values[valid]) / np.sum(pop_values[valid])
        pop_weighted_records.append(weighted_mean)
        
    # Store results in admin1_records
    for i, row in enumerate(admin2_shp.itertuples()):
        admin2_records.append({
            "admin2": row.admin2,
            "year": year,
            "surveillance_nonweighted": surveillance_nonweighted[i]["mean"],
            "surveillance_pop_weighted": pop_weighted_records[i]
        })

# Convert to DataFrames
df_admin1_surveillance = pd.DataFrame(admin1_records)
df_admin2_surveillance = pd.DataFrame(admin2_records)

# Optional: reset index
df_admin1_surveillance = df_admin1_surveillance.reset_index(drop=True)
df_admin2_surveillance = df_admin2_surveillance.reset_index(drop=True)

In [15]:
df_admin2_surveillance

,admin2,year,surveillance_nonweighted,surveillance_pop_weighted
0,ACEH BARAT,2015,0.369001,0.639564
1,ACEH BARAT DAYA,2015,0.297746,0.740813
2,ACEH BESAR,2015,0.510617,0.858704
3,ACEH JAYA,2015,0.236977,0.572931
4,ACEH SELATAN,2015,0.233088,0.498598
...,...,...,...,...
5135,WAY KANAN,2024,0.515941,0.606863
5136,WONOGIRI,2024,0.838876,0.870500
5137,WONOSOBO,2024,0.848702,0.882246
5138,YAHUKIMO,2024,0.057116,0.077861


In [14]:
out_folder = os.path.join(folder, "admin_year_surveillance")
os.makedirs(out_folder, exist_ok=True)

df_admin1_surveillance.to_csv(os.path.join(out_folder, "admin1_year_surveillance.csv"), index=False)
df_admin2_surveillance.to_csv(os.path.join(out_folder, "admin2_year_surveillance.csv"), index=False)

---

### urban surveillance

In [29]:
admin1_records = []
admin2_records = []
raster_urbanisation_binary_pop_aligned_mask = raster_urbanisation_binary_pop_aligned.where(raster_urbanisation_binary_pop_aligned==1, other=np.nan)
raster_urban_surveillance_pop_aligned = raster_surveillance_pop_aligned * raster_urbanisation_binary_pop_aligned_mask
raster_urban_surveillance_pop_aligned = raster_urban_surveillance_pop_aligned.rio.write_nodata(np.nan, inplace=False)

In [88]:
for year in range(2015,2025):
    pop_crop = raster_pop[year].squeeze().rio.clip_box(minx=region_bounds_buffered[0], miny=region_bounds_buffered[1],
                                                   maxx=region_bounds_buffered[2], maxy=region_bounds_buffered[3])
    # admin1
    # Non-weighted: simple mean over each polygon
    urban_surveillance_nonweighted = zonal_stats(
        vectors=admin1_shp,
        raster=raster_urban_surveillance_pop_aligned.values,
        affine=raster_urban_surveillance_pop_aligned.rio.transform(),
        stats=["mean"],
        nodata=raster_urban_surveillance_pop_aligned.rio.nodata,
        all_touched=True
    )

    # Population-weighted: sum(surveillance * pop) / sum(pop)
    urban_surveillance_pop_weighted_records = []
    for i, row in enumerate(admin1_shp.itertuples()):
        mask = rasterize([(row.geometry, 1)],
                         out_shape=raster_urban_surveillance_pop_aligned.shape[-2:],
                         transform=raster_urban_surveillance_pop_aligned.rio.transform(),
                         fill=0,
                         all_touched=True,
                         dtype="uint8"
                         ).astype(bool)
        urban_surveillance_values = raster_urban_surveillance_pop_aligned.values[mask]
        pop_values = pop_crop.values[mask]

        # Keep only cells where BOTH are valid
        valid = (~np.isnan(urban_surveillance_values)) & (~np.isnan(pop_values)) & (pop_values > 0)

        if not np.any(valid):
            weighted_mean = np.nan
        else:
            weighted_mean = np.sum(urban_surveillance_values[valid] * pop_values[valid]) / np.sum(pop_values[valid])
        urban_surveillance_pop_weighted_records.append(weighted_mean)
        
    # Store results in admin1_records
    for i, row in enumerate(admin1_shp.itertuples()):
        admin1_records.append({
            "admin1": row.admin1,
            "year": year,
            "urban_surveillance_nonweighted": urban_surveillance_nonweighted[i]["mean"],
            "urban_surveillance_pop_weighted": urban_surveillance_pop_weighted_records[i]
        })

    # admin2
    # Non-weighted: simple mean over each polygon
    urban_surveillance_nonweighted = zonal_stats(
        vectors=admin2_shp,
        raster=raster_urban_surveillance_pop_aligned.values,
        affine=raster_urban_surveillance_pop_aligned.rio.transform(),
        stats=["mean"],
        nodata=raster_urban_surveillance_pop_aligned.rio.nodata,
        all_touched=True
    )

    # Population-weighted: sum(surveillance * pop) / sum(pop)
    urban_surveillance_pop_weighted_records = []
    for i, row in enumerate(admin2_shp.itertuples()):
        mask = rasterize([(row.geometry, 1)],
                         out_shape=raster_urban_surveillance_pop_aligned.shape[-2:],
                         transform=raster_urban_surveillance_pop_aligned.rio.transform(),
                         fill=0,
                         all_touched=True,
                         dtype="uint8"
                         ).astype(bool)
        urban_surveillance_values = raster_urban_surveillance_pop_aligned.values[mask]
        pop_values = pop_crop.values[mask]

        # Keep only cells where BOTH are valid
        valid = (~np.isnan(urban_surveillance_values)) & (~np.isnan(pop_values)) & (pop_values > 0)

        if not np.any(valid):
            weighted_mean = np.nan
        else:
            weighted_mean = np.sum(urban_surveillance_values[valid] * pop_values[valid]) / np.sum(pop_values[valid])
        urban_surveillance_pop_weighted_records.append(weighted_mean)
        
    # Store results in admin2_records
    for i, row in enumerate(admin2_shp.itertuples()):
        admin2_records.append({
            "admin2": row.admin2,
            "year": year,
            "urban_surveillance_nonweighted": urban_surveillance_nonweighted[i]["mean"],
            "urban_surveillance_pop_weighted": urban_surveillance_pop_weighted_records[i]
        })

# Convert to DataFrames
df_admin1_urban_surveillance = pd.DataFrame(admin1_records)
df_admin2_urban_surveillance = pd.DataFrame(admin2_records)

# Optional: reset index
df_admin1_urban_surveillance = df_admin1_urban_surveillance.reset_index(drop=True)
df_admin2_urban_surveillance = df_admin2_urban_surveillance.reset_index(drop=True)

In [89]:
df_admin2_urban_surveillance

,admin2,year,urban_surveillance_nonweighted,urban_surveillance_pop_weighted
0,ACEH BARAT,2015,0.829057,0.787234
1,ACEH BARAT DAYA,2015,0.825269,0.844500
2,ACEH BESAR,2015,0.915559,0.939699
3,ACEH JAYA,2015,0.864509,0.879474
4,ACEH SELATAN,2015,0.694324,0.680381
...,...,...,...,...
5649,WAY KANAN,2024,0.765571,0.760155
5650,WONOGIRI,2024,0.868872,0.888670
5651,WONOSOBO,2024,0.864985,0.891884
5652,YAHUKIMO,2024,0.171283,0.184696


In [90]:
out_folder = os.path.join(folder, "admin_year_urban_surveillance")
os.makedirs(out_folder, exist_ok=True)

df_admin1_surveillance.to_csv(os.path.join(out_folder, "admin1_year_urban_surveillance.csv"), index=False)
df_admin2_surveillance.to_csv(os.path.join(out_folder, "admin2_year_urban_surveillance.csv"), index=False)

---

### climate variables

In [6]:
weather_folder = os.path.join(folder, "weather_statistics-(reanalysis-era5-land)")

In [7]:
admin1_records = []
admin2_records = []

In [64]:
t2m = xr.open_dataset(os.path.join(weather_folder, f"t2m/t2m_{year}.grib"),engine="cfgrib")
t2m = t2m.isel(time=slice(1, None))
t2m = t2m['t2m']
t2m = t2m.rio.write_crs("EPSG:4326")
# t2m

In [65]:
tp = xr.open_dataset(os.path.join(weather_folder, f"tp/tp_{year}.grib"),engine="cfgrib")
tp = tp.isel(time=slice(1, None))
tp = tp['tp']
tp = tp.rio.write_crs("EPSG:4326")
# tp

In [66]:
rh = xr.open_dataset(os.path.join(weather_folder, f"rh/rh_{year}.nc"),engine="netcdf4")
rh = rh['rh']
rh = rh.rio.write_crs("EPSG:4326")
# rh

In [8]:
admin1_dict = {}
admin2_dict = {}

In [9]:
t2m_monthly_mean = {}
t2m_monthly_min = {}
t2m_monthly_max = {}
tp_monthly_mean = {}
rh_monthly_mean = {}

pixel_monthly_statistics = {}

t2m_monthly_mean_aligned = {}
t2m_monthly_min_aligned = {}
t2m_monthly_max_aligned = {}
tp_monthly_mean_aligned = {}
rh_monthly_mean_aligned = {}

pixel_monthly_statistics_aligned = {}

stat_names = ["t2m_mean", "t2m_min", "t2m_max", "tp_mean", "rh_mean"]

In [10]:
for year in range(2015,2025):
    # read in climate data datasets
    t2m = xr.open_dataset(os.path.join(weather_folder, f"t2m/t2m_{year}.grib"),engine="cfgrib")
    t2m = t2m.isel(time=slice(1, None))
    t2m = t2m['t2m']
    t2m = t2m.rio.write_crs("EPSG:4326")
    tp = xr.open_dataset(os.path.join(weather_folder, f"tp/tp_{year}.grib"),engine="cfgrib")
    tp = tp.isel(time=slice(1, None))
    tp = tp['tp']
    tp = tp.rio.write_crs("EPSG:4326")
    rh = xr.open_dataset(os.path.join(weather_folder, f"rh/rh_{year}.nc"),engine="netcdf4")
    rh = rh['rh']
    rh = rh.rio.write_crs("EPSG:4326")

    # calculate monthly pixel statistics
    t2m['time'] = pd.to_datetime(t2m['time'].values)
    t2m_monthly_mean[year] = t2m.groupby('time.month').mean(dim=['time', 'step'])
    t2m_monthly_min[year] = t2m.groupby('time.month').min(dim=['time', 'step'])
    t2m_monthly_max[year] = t2m.groupby('time.month').max(dim=['time', 'step'])

    tp['time'] = pd.to_datetime(tp['time'].values)
    tp_monthly_mean[year] = tp.groupby('time.month').mean(dim=['time', 'step'])

    rh['time'] = pd.to_datetime(rh['time'].values)
    rh_monthly_mean[year] = rh.groupby('time.month').mean(dim=['time', 'step'])

    # align
    align_methods = {"t2m_mean": "bilinear",
                     "t2m_min":  "nearest",
                     "t2m_max":  "nearest",
                     "tp_mean":  "bilinear",
                     "rh_mean":  "bilinear"}
    
    _, t2m_monthly_mean_aligned[year] = best_res_align(pop_crop_alignment, '',
                                            t2m_monthly_mean[year], align_methods['t2m_mean'],
                                            region_bounds_buffered = region_bounds_buffered,
                                            shape_crs = admin1_shp.crs)
    _, t2m_monthly_min_aligned[year] = best_res_align(pop_crop_alignment, '',
                                            t2m_monthly_min[year], align_methods['t2m_min'],
                                            region_bounds_buffered = region_bounds_buffered,
                                            shape_crs = admin1_shp.crs)
    _, t2m_monthly_max_aligned[year] = best_res_align(pop_crop_alignment, '',
                                            t2m_monthly_max[year], align_methods['t2m_max'],
                                            region_bounds_buffered = region_bounds_buffered,
                                            shape_crs = admin1_shp.crs)
    _, tp_monthly_mean_aligned[year] = best_res_align(pop_crop_alignment, '',
                                            tp_monthly_mean[year], align_methods['tp_mean'],
                                            region_bounds_buffered = region_bounds_buffered,
                                            shape_crs = admin1_shp.crs)
    _, rh_monthly_mean_aligned[year] = best_res_align(pop_crop_alignment, '',
                                            rh_monthly_mean[year], align_methods['rh_mean'],
                                            region_bounds_buffered = region_bounds_buffered,
                                            shape_crs = admin1_shp.crs)

pixel_monthly_statistics = {"t2m_mean" : t2m_monthly_mean,
                            "t2m_min" : t2m_monthly_min,
                            "t2m_max" : t2m_monthly_max,
                            "tp_mean" : tp_monthly_mean,
                            "rh_mean" : rh_monthly_mean}
pixel_monthly_statistics_aligned = {"t2m_mean" : t2m_monthly_mean_aligned,
                                    "t2m_min" : t2m_monthly_min_aligned,
                                    "t2m_max" : t2m_monthly_max_aligned,
                                    "tp_mean" : tp_monthly_mean_aligned,
                                    "rh_mean" : rh_monthly_mean_aligned}

In [18]:
admin1_dict = {}
admin2_dict = {}
for stat_name in stat_names[:]:
    print(stat_name)
    for year in range(2015,2017):
        print(year)
        pixel_statistic_aligned = pixel_monthly_statistics_aligned[stat_name][year]

        # admin1
        statistic_unweighted_records_1 = []
        statistic_pop_weighted_records_1 = []

        for i, row in enumerate(admin1_shp.itertuples()):
            if i>1:
                break
            print(row.admin1)
            mask = rasterize([(row.geometry, 1)],
                                out_shape=pixel_statistic_aligned.shape[-2:],
                                transform=pixel_statistic_aligned.rio.transform(),
                                fill=0,
                                all_touched=True,
                                dtype="uint8"
                                ).astype(bool)
            statistic_values = np.where(mask, pixel_statistic_aligned.values, np.nan)
            pop_values = np.where(mask, pop_crop_alignment.values, np.nan)
            pop_values_b = np.broadcast_to(pop_values, statistic_values.shape)
            if np.all(np.isnan(statistic_values)):
                print('all nan')
                continue
            unweighted_mean_per_month = np.nanmean(statistic_values, axis=(1,2))
            statistic_unweighted_records_1.append(unweighted_mean_per_month)

            valid = (~np.isnan(statistic_values)) & (~np.isnan(pop_values_b))
            weighted_sum = np.nansum(statistic_values * pop_values_b * valid, axis=(1,2))
            pop_sum = np.nansum(pop_values_b * valid, axis=(1,2))
            
            pop_weighted_mean_per_month = weighted_sum / pop_sum
            statistic_pop_weighted_records_1.append(pop_weighted_mean_per_month)

            for month in range(1, statistic_values.shape[0]+1):
                key = (row.admin1, year, month)
                if key not in admin1_dict:
                    admin1_dict[key] = {
                        "admin1": row.admin1,
                        "year": year,
                        "month": month
                    }
                admin1_dict[key][f"{stat_name}_unweighted"] = unweighted_mean_per_month[month-1]
                admin1_dict[key][f"{stat_name}_pop_weighted"] = pop_weighted_mean_per_month[month-1]

        # admin2
        statistic_unweighted_records_2 = []
        statistic_pop_weighted_records_2 = []

        for i, row in enumerate(admin2_shp.itertuples()):
            if i>1:
                break
            print(row.admin2)
            mask = rasterize([(row.geometry, 1)],
                                out_shape=pixel_statistic_aligned.shape[-2:],
                                transform=pixel_statistic_aligned.rio.transform(),
                                fill=0,
                                all_touched=True,
                                dtype="uint8"
                                ).astype(bool)
            statistic_values = np.where(mask, pixel_statistic_aligned.values, np.nan)
            pop_values = np.where(mask, pop_crop_alignment.values, np.nan)
            pop_values_b = np.broadcast_to(pop_values, statistic_values.shape)
            if np.all(np.isnan(statistic_values)):
                print('all nan')
                continue
            unweighted_mean_per_month = np.nanmean(statistic_values, axis=(1,2))
            statistic_unweighted_records_2.append(unweighted_mean_per_month)

            valid = (~np.isnan(statistic_values)) & (~np.isnan(pop_values_b))
            weighted_sum = np.nansum(statistic_values * pop_values_b * valid, axis=(1,2))
            pop_sum = np.nansum(pop_values_b * valid, axis=(1,2))
            
            pop_weighted_mean_per_month = weighted_sum / pop_sum
            statistic_pop_weighted_records_2.append(pop_weighted_mean_per_month)

            for month in range(1, statistic_values.shape[0]+1):
                key = (row.admin2, year, month)
                if key not in admin2_dict:
                    admin2_dict[key] = {
                        "admin2": row.admin2,
                        "year": year,
                        "month": month
                    }
                admin2_dict[key][f"{stat_name}_unweighted"] = unweighted_mean_per_month[month-1]
                admin2_dict[key][f"{stat_name}_pop_weighted"] = pop_weighted_mean_per_month[month-1]

t2m_mean
2015
ACEH
BALI
ACEH BARAT
ACEH BARAT DAYA
2016
ACEH
BALI
ACEH BARAT
ACEH BARAT DAYA
t2m_min
2015
ACEH
BALI
ACEH BARAT
ACEH BARAT DAYA
2016
ACEH
BALI
ACEH BARAT
ACEH BARAT DAYA
t2m_max
2015
ACEH
BALI
ACEH BARAT
ACEH BARAT DAYA
2016
ACEH
BALI
ACEH BARAT
ACEH BARAT DAYA
tp_mean
2015
ACEH
BALI
ACEH BARAT
ACEH BARAT DAYA
2016
ACEH
BALI
ACEH BARAT
ACEH BARAT DAYA
rh_mean
2015
ACEH
BALI
ACEH BARAT
ACEH BARAT DAYA
2016
ACEH
BALI
ACEH BARAT
ACEH BARAT DAYA


In [19]:
# Convert to DataFrame
admin1_records = list(admin1_dict.values())
df_admin1_statistics = pd.DataFrame(admin1_records)
df_admin1_statistics = df_admin1_statistics.sort_values(by=["admin1", "year", "month"]).reset_index(drop=True)

admin2_records = list(admin2_dict.values())
df_admin2_statistics = pd.DataFrame(admin2_records)
df_admin2_statistics = df_admin2_statistics.sort_values(by=["admin2", "year", "month"]).reset_index(drop=True)

In [16]:
df_admin1_statistics.to_csv(os.path.join(folder, 'df_admin1_statistics.csv'), index=False)
df_admin2_statistics.to_csv(os.path.join(folder, 'df_admin2_statistics.csv'), index=False)

---

In [46]:
f_admin1_dict = {}
f_admin2_dict = {}
for stat_name in stat_names[:2]:
    print(stat_name)
    
    years = list(range(2015, 2017))
    pixel_statistic_stacked = xr.concat(
        [pixel_monthly_statistics_aligned[stat_name][y] for y in years],
        dim="year")
    pixel_statistic_stacked = pixel_statistic_stacked.assign_coords(year=years)

    # admin1
    for i, row in enumerate(admin1_shp.itertuples()):
        if i>1:
            break
        print(row.admin1)
        mask = rasterize([(row.geometry, 1)],
                            out_shape=pixel_statistic_stacked.shape[-2:],
                            transform=pixel_statistic_stacked.rio.transform(),
                            fill=0,
                            all_touched=True,
                            dtype="uint8"
                            ).astype(bool)
        statistic_values = np.where(mask, pixel_statistic_stacked.values, np.nan)
        pop_values = np.where(mask, pop_crop_alignment.values, np.nan)
        pop_values_b = np.broadcast_to(pop_values, statistic_values.shape)
        
        unweighted_mean_per_month = np.nanmean(statistic_values, axis=(2,3))

        valid = (~np.isnan(statistic_values)) & (~np.isnan(pop_values_b))
        weighted_sum = np.nansum(statistic_values * pop_values_b * valid, axis=(2,3))
        pop_sum = np.nansum(pop_values_b * valid, axis=(2,3))
        
        pop_weighted_mean_per_month = weighted_sum / pop_sum

        for year in range(2015, statistic_values.shape[0]+2015):
            for month in range(1, statistic_values.shape[1]+1):
                key = (row.admin1, year, month)
                if key not in f_admin1_dict:
                    f_admin1_dict[key] = {
                        "admin1": row.admin1,
                        "year": year,
                        "month": month
                    }
                f_admin1_dict[key][f"{stat_name}_unweighted"] = unweighted_mean_per_month[year-2015, month-1]
                f_admin1_dict[key][f"{stat_name}_pop_weighted"] = pop_weighted_mean_per_month[year-2015, month-1]

t2m_mean
ACEH
BALI
t2m_min
ACEH
BALI


In [66]:
f_admin1_dict = {}
f_admin2_dict = {}

# concatenate datasets
years = list(range(2015, 2017))
pixel_statistic_stacked = xr.concat(
    [pixel_monthly_statistics_aligned[stat_name][y] for stat_name in stat_names[:2] for y in years],
    dim="month")

# admin1
for i, row in enumerate(admin1_shp.itertuples()):
    if i>1:
        break
    print(row.admin1)
    mask = rasterize([(row.geometry, 1)],
                        out_shape=pixel_statistic_stacked.shape[-2:],
                        transform=pixel_statistic_stacked.rio.transform(),
                        fill=0,
                        all_touched=True,
                        dtype="uint8"
                        ).astype(bool)
    statistic_values = np.where(mask, pixel_statistic_stacked.values, np.nan)
    print(statistic_values.shape)
    pop_values = np.where(mask, pop_crop_alignment.values, np.nan)
    pop_values_b = np.broadcast_to(pop_values, statistic_values.shape)
    
    unweighted_mean_per_month = np.nanmean(statistic_values, axis=(1,2))

    valid = (~np.isnan(statistic_values)) & (~np.isnan(pop_values_b))
    weighted_sum = np.nansum(statistic_values * pop_values_b * valid, axis=(1,2))
    pop_sum = np.nansum(pop_values_b * valid, axis=(1,2))
    
    pop_weighted_mean_per_month = weighted_sum / pop_sum

    for i in range(0, statistic_values.shape[0]):
        stat_name = stat_names[i//(len(years)*12)]
        year = (i - (i//(len(years)*12))*(len(years)*12)) // 12 + 2015
        month = (i % 12) + 1
        key = (row.admin1, year, month)
        if key not in f_admin1_dict:
            f_admin1_dict[key] = {
                "admin1": row.admin1,
                "year": year,
                "month": month
            }
        f_admin1_dict[key][f"{stat_name}_unweighted"] = unweighted_mean_per_month[i]
        f_admin1_dict[key][f"{stat_name}_pop_weighted"] = pop_weighted_mean_per_month[i]

(48, 2094, 5565)
ACEH
(48, 2094, 5565)
BALI
(48, 2094, 5565)


In [72]:
f_admin1_dict = {}
f_admin2_dict = {}

# concatenate datasets
pixel_statistic_stacked = xr.concat(
    [
        xr.concat(
            [pixel_monthly_statistics_aligned[s][y] for y in years],
            dim="year"
        ).assign_coords(year=years)
        for s in stat_names[:2]
    ],
    dim="stat"
).assign_coords(stat=stat_names[:2])

# admin1
for i, row in enumerate(admin1_shp.itertuples()):
    if i>1:
        break
    print(row.admin1)
    mask = rasterize([(row.geometry, 1)],
                        out_shape=pixel_statistic_stacked.shape[-2:],
                        transform=pixel_statistic_stacked.rio.transform(),
                        fill=0,
                        all_touched=True,
                        dtype="uint8"
                        ).astype(bool)
    statistic_values = np.where(mask, pixel_statistic_stacked.values, np.nan)
    print(statistic_values.shape)
    pop_values = np.where(mask, pop_crop_alignment.values, np.nan)
    pop_values_b = np.broadcast_to(pop_values, statistic_values.shape)
    
    unweighted_mean_per_month = np.nanmean(statistic_values, axis=(3,4))

    valid = (~np.isnan(statistic_values)) & (~np.isnan(pop_values_b))
    weighted_sum = np.nansum(statistic_values * pop_values_b * valid, axis=(3,4))
    pop_sum = np.nansum(pop_values_b * valid, axis=(3,4))
    
    pop_weighted_mean_per_month = weighted_sum / pop_sum
    for stat_i in range(statistic_values.shape[0]):
        stat_name = stat_names[stat_i]
        for year in range(2015, statistic_values.shape[1]+2015):
            for month in range(1, statistic_values.shape[2]+1):
                key = (row.admin1, year, month)
                if key not in f_admin1_dict:
                    f_admin1_dict[key] = {
                        "admin1": row.admin1,
                        "year": year,
                        "month": month
                    }
                f_admin1_dict[key][f"{stat_name}_unweighted"] = unweighted_mean_per_month[stat_i, year-2015, month-1]
                f_admin1_dict[key][f"{stat_name}_pop_weighted"] = pop_weighted_mean_per_month[stat_i, year-2015, month-1]

ACEH
(2, 2, 12, 2094, 5565)
BALI
(2, 2, 12, 2094, 5565)


In [ ]:
f_admin1_dict = {}
f_admin2_dict = {}

# concatenate datasets
years = range(2015, 2025)
pixel_statistic_stacked = xr.concat(
    [
        xr.concat(
            [pixel_monthly_statistics_aligned[s][y] for y in years],
            dim="year"
        ).assign_coords(year=years)
        for s in stat_names[:]
    ],
    dim="stat"
).assign_coords(stat=stat_names[:])

# admin1
for i, row in enumerate(admin1_shp.itertuples()):
    print(row.admin1)
    mask = rasterize([(row.geometry, 1)],
                        out_shape=pixel_statistic_stacked.shape[-2:],
                        transform=pixel_statistic_stacked.rio.transform(),
                        fill=0,
                        all_touched=True,
                        dtype="uint8"
                        ).astype(bool)
    mask_da = xr.DataArray(
        mask,
        dims=("y", "x"),
        coords=pixel_statistic_stacked.isel(stat=0, year=0, month=0).coords
    )

    stat_m = pixel_statistic_stacked.where(mask_da)
    pop_m = pop_crop_alignment.where(mask_da)

    # ---- vectorised across stat, year, month ----
    unweighted = stat_m.mean(dim=("y", "x"))
    weighted = (stat_m * pop_m).sum(dim=("y", "x")) / pop_m.sum(dim=("y", "x"))

    # ---- write results ----
    for stat in stat_m.stat.values:
        for year in stat_m.year.values:
            for month in stat_m.month.values:
                key = (row.admin1, int(year), int(month))

                d = f_admin1_dict.setdefault(
                    key,
                    {"admin1": row.admin1, "year": int(year), "month": int(month)}
                )

                d[f"{stat}_unweighted"] = float(unweighted.sel(stat=stat, year=year, month=month))
                d[f"{stat}_pop_weighted"] = float(weighted.sel(stat=stat, year=year, month=month))
# admin2
for i, row in enumerate(admin2_shp.itertuples()):
    print(row.admin2)
    mask = rasterize([(row.geometry, 1)],
                        out_shape=pixel_statistic_stacked.shape[-2:],
                        transform=pixel_statistic_stacked.rio.transform(),
                        fill=0,
                        all_touched=True,
                        dtype="uint8"
                        ).astype(bool)
    mask_da = xr.DataArray(
        mask,
        dims=("y", "x"),
        coords=pixel_statistic_stacked.isel(stat=0, year=0, month=0).coords
    )

    stat_m = pixel_statistic_stacked.where(mask_da)
    pop_m = pop_crop_alignment.where(mask_da)

    # ---- vectorised across stat, year, month ----
    unweighted = stat_m.mean(dim=("y", "x"))
    weighted = (stat_m * pop_m).sum(dim=("y", "x")) / pop_m.sum(dim=("y", "x"))

    # ---- write results ----
    for stat in stat_m.stat.values:
        for year in stat_m.year.values:
            for month in stat_m.month.values:
                key = (row.admin2, int(year), int(month))

                d = f_admin2_dict.setdefault(
                    key,
                    {"admin2": row.admin2, "year": int(year), "month": int(month)}
                )

                d[f"{stat}_unweighted"] = float(unweighted.sel(stat=stat, year=year, month=month))
                d[f"{stat}_pop_weighted"] = float(weighted.sel(stat=stat, year=year, month=month))

ACEH


In [ ]:
# Convert to DataFrame
f_admin1_records = list(f_admin1_dict.values())
f_df_admin1_statistics = pd.DataFrame(f_admin1_records)
f_df_admin1_statistics = f_df_admin1_statistics.sort_values(by=["admin1", "year", "month"]).reset_index(drop=True)

f_admin2_records = list(f_admin2_dict.values())
f_df_admin2_statistics = pd.DataFrame(f_admin2_records)
f_df_admin2_statistics = f_df_admin2_statistics.sort_values(by=["admin2", "year", "month"]).reset_index(drop=True)

In [75]:
f_df_admin1_statistics.shape

(48, 7)

In [76]:
f_df_admin1_statistics.columns

Index(['admin1', 'year', 'month', 't2m_mean_unweighted',
       't2m_mean_pop_weighted', 't2m_min_unweighted', 't2m_min_pop_weighted'],
      dtype='object')

In [44]:
df_admin1_statistics.columns

Index(['admin1', 'year', 'month', 't2m_mean_unweighted',
       't2m_mean_pop_weighted', 't2m_min_unweighted', 't2m_min_pop_weighted',
       't2m_max_unweighted', 't2m_max_pop_weighted', 'tp_mean_unweighted',
       'tp_mean_pop_weighted', 'rh_mean_unweighted', 'rh_mean_pop_weighted'],
      dtype='object')

In [82]:
all(f_df_admin2_statistics == df_admin2_statistics[['admin2', 'year', 'month', 't2m_mean_unweighted',
       't2m_mean_pop_weighted', 't2m_min_unweighted', 't2m_min_pop_weighted']])

True